# POPCON study

In [ ]:
using Plots
using FUSE
FUSE.ProgressMeter.ijulia_behavior(:clear);

### Setup distributed computing environment

In [ ]:
using Distributed
FUSE.parallel_environment("localhost", 8)
@everywhere using FUSE

### Define workflow

In [ ]:
@everywhere function workflow(T_scale::Float64, n_scale::Float64)
    dd = IMAS.dd()

    ini, act = FUSE.case_parameters(:ITER; init_from=:scalars)
    
    ini.core_profiles.ne_setting = :greenwald_fraction
    ini.core_profiles.ne_value = 0.8
    
    ini.core_profiles.ne_value *= n_scale
    ini.equilibrium.pressure_core *= T_scale * n_scale
    
    ini.hcd.power_scaling_cost_function = dd -> abs(1.0 - IMAS.tau_e_thermal(dd) / IMAS.tau_e_h98(dd))

    try
        FUSE.init(dd, ini, act)
    catch e
        println(e)
    finally
        return dd
    end
end

## Run nominal conditions

In [ ]:
FUSE.logging(Logging.Info; actors=Logging.Info)
dd = workflow(1.0, 1.0)
nominal = IMAS.extract(dd)
#FUSE.digest(dd)

## Extent of the scan

In [ ]:
T_scale_ = LinRange(10, 25, 5) ./ nominal[:Te0].value # [keV]
n_scale_ = LinRange(0.2, 1.2, 5) ./ nominal[:fGW].value # [fGW]
cases = [[T_scale, n_scale] for T_scale in T_scale_, n_scale in n_scale_]

### Run simulations

In [ ]:
@everywhere FUSE.logging(Logging.Info; actors=Logging.Error)
DD = FUSE.ProgressMeter.@showprogress pmap(case -> workflow(case...), cases);

### Extract quantities of interest from series of `dd`'s

In [ ]:
results = FUSE.extract(reshape(DD, length(DD)); filter_invalid=:none);

### POPCON plots

In [ ]:
all_plots = []
for key in [:τe, :Pfusion, :Qfusion, :Paux_tot, :Prad_tot, :Psol]
    results_ = reshape(getproperty(results, key), (length(T_scale_), length(n_scale_)))
    p = contourf(T_scale_.*nominal[:Te0].value, n_scale_.*nominal[:fGW].value, abs.(results_'), levels=11, alpha=0.5)
    plot!(p, grid=true, xlabel="T [kev]", ylabel="fGW", title="$(key) [$(nominal[key].units)]")
    push!(all_plots,p)
end
plot(all_plots..., size=(1200,550), bottom_margin=5 * Plots.Measures.mm, left_margin=5 * Plots.Measures.mm,)

## In depth view of one of the solutions

In [ ]:
FUSE.digest(DD[4,2])